# Video Games Sales

This notebook is for data analysis of video game sales.

Data: [Kaggle.com](https://www.kaggle.com/datasets/gregorut/videogamesales)

In [55]:
import pandas as pd

# Load the dataset from local storage and do the basic checks

Firstly, we will load the data into df_main and then we will display some basic information about the data frame - the head and information.

In [56]:
df_main = pd.read_csv('../files/vgsales.csv')
df_main.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [57]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


## Missing Values

THere are couple of columns with missing data:
1. Year
2. Publisher

Let's verify for which games we are missing this data. If these are blockbusters, we need to do something about those. But if they are some unknown titles for old platforms, we can drop observations with values missing in these columns

In [58]:
df_main[df_main['Year'].isna()]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
179,180,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23
377,378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49
431,432,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17
470,471,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,NaN,1.57,1.02,0.00,0.41,3.00
607,608,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0.00,0.03,2.53
...,...,...,...,...,...,...,...,...,...,...,...
16307,16310,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0.00,0.00,0.00,0.01
16327,16330,Inversion,PC,NaN,Shooter,Namco Bandai Games,0.01,0.00,0.00,0.00,0.01
16366,16369,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,Unknown,0.01,0.00,0.00,0.00,0.01
16427,16430,Virtua Quest,GC,NaN,Role-Playing,Unknown,0.01,0.00,0.00,0.00,0.01


As we can see, there are some titles with significant sale result (like LEGO: BATMAN for Wii or FIFA Soccer 2004 for PS2).

Some are not important, so let's filter for those which have sales bigger than 0.5

In [59]:
df_games_with_missing_year = df_main[(df_main['Year'].isna()) & (df_main['Global_Sales'] > 0.5)]

There are some significant releases

We need to find release year for them.

I will [Twitch API](https://api-docs.igdb.com/#getting-started) to do it

In [63]:
import os
import requests
import asyncio
import aiohttp
import ssl
from dotenv import load_dotenv

# load .env file
load_dotenv()

# read Twitch API credentials from .env file
twitch_client_id = os.getenv('TWITCH_CLIENT_ID')
twitch_client_secret = os.getenv('TWITCH_CLIENT_SECRET')

# prepare request
url = f'https://id.twitch.tv/oauth2/token?client_id={twitch_client_id}&client_secret={twitch_client_secret}&grant_type=client_credentials'

# send request for token
response = requests.post(url)

# extract token
access_token = response.json()['access_token']

In [64]:
twitch_api_url = 'https://api.igdb.com/v4'

headers = {
    'Client-ID': twitch_client_id,
    'Authorization': f'Bearer {access_token}'
}
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [67]:
async def get_platform(abbreviation):
    url = f'{twitch_api_url}/platforms'
    async with aiohttp.ClientSession() as session:
        async with session.post(url, 
                                headers=headers, 
                                data=f'fields *; where abbreviation = "{abbreviation}";',
                                ssl=ssl_context) as response:
            return await response.json()

platform_abbreviations = df_games_with_missing_year['Platform'].unique()

platform = await get_platform('PS4')
platform

[{'id': 48,
  'abbreviation': 'PS4',
  'alternative_name': 'PS4',
  'category': 1,
  'created_at': 1326544944,
  'generation': 8,
  'name': 'PlayStation 4',
  'platform_logo': 231,
  'platform_family': 1,
  'slug': 'ps4--1',
  'summary': 'The PlayStation 4 system opens the door to an incredible journey through immersive new gaming worlds and a deeply connected gaming community. PS4 puts gamers first with an astounding launch line-up and over 180 games in development. Play amazing top-tier blockbusters and innovative indie hits on PS4. Developer inspired, gamer focused.',
  'updated_at': 1718027289,
  'url': 'https://www.igdb.com/platforms/ps4--1',
  'versions': [17, 178, 179],
  'websites': [11],
  'checksum': '1dfa9b83-5f1d-56ce-01a1-06fcdfbb5bc0'}]

In [34]:


response = requests.post(f'{twitch_api_url}/platforms', headers=headers, data='fields *; where abbreviation = "PS3";')

print([x['id'] for x in response.json()])

[9]


In [40]:
query = f'search "Yakuza 4"; fields release_dates; where platforms = (9);'

response = requests.post(f'{twitch_api_url}/games', headers=headers, data=query)

print(response.json())

[{'id': 2062, 'release_dates': [16192, 107005, 107006, 107007]}]


In [43]:
query='fields *; where id = 16192;'

response = requests.post(f'{twitch_api_url}/release_dates', headers=headers, data=query)

print(response.json())

[{'id': 16192, 'category': 0, 'created_at': 1399445605, 'date': 1268006400, 'game': 2062, 'human': 'Mar 08, 2010', 'm': 3, 'platform': 9, 'region': 8, 'updated_at': 1709835999, 'y': 2010, 'checksum': '95f05931-75f2-abe4-d96e-67a0a88527a3'}]
